In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
import torch
!pip install rouge_score

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlptweets/NLPData_chatGPT.csv
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=b6dddcf02d9eb8448a27569bb15bbadc903fb46a7c79700ad41c78bff7b39117
  Stored in directory: /root/.cache/pip/wheels/8e/6b/70/59daa7c90a238610e34bac5916e001fe3d9bb0ec59c8cf5518
Successfully built rouge_score


In [1]:
! pip install simplet5 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.12.2 requires cupy-cuda115, which is not installed.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
distributed 2021.11.2 requires dask==2021.11.2, but you have dask 2022.2.0 which is incompatible.
dask-cudf 21.12.2 requires dask<=2021.11.2,>=2021.11.1, but you have dask 2022.2.0 which is incompatible.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have

In [3]:
# pip install rouge_score

In [2]:
import pandas as pd
df=pd.read_csv("/kaggle/input/nlptweets/NLPData_chatGPT.csv")

## Changing Dataset to suitable form
#### T5 need columns as source_text and target_text

In [3]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"y":"target_text", "x":"source_text"})
df = df[['source_text', 'target_text']]
df

,source_text,target_text
0,Summarise the given tweets in English: \nAhead...,\n\nPeople have appealed to accept the judgeme...
1,Summarise the given tweets in English: \nWATCH...,\n\nAt a meeting at NSA Ajit Doval's residence...
2,Summarise the given tweets in English: \nA civ...,.\n\nThe AyodhyaVerdict has finally been reach...
3,Summarise the given tweets in English: \nAs a ...,"\n\nThe tweets discuss the Ayodhya Verdict, a ..."
4,Summarise the given tweets in English: \nI wou...,\n\nThis series of tweets discusses the recent...
...,...,...
1152,Summarise the given tweets in English: \nintut...,\n\nA 19 year old Muthyam from Maharashtra die...
1153,Summarise the given tweets in English: \nOn Ca...,"OUS\n\nA 19-year-old man from Maharashtra, Mut..."
1154,Summarise the given tweets in English: \nI had...,\n\nPeople have had doubts about the Covid-19 ...
1155,Summarise the given tweets in English: \nanxie...,\n\nPeople are feeling overwhelmed with the am...


In [4]:
for i in range(len(df)):
    df['source_text'][i]=df['source_text'][i][40:]

In [35]:
# for i in range(len(df)):
#     if(len(df['source_text'][i])>512):
#         df['source_text'][i]=df['source_text'][i][0:512]
    

In [5]:
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,"summarize: Ahead of AyodhyaVerdict, appeal to ...",\n\nPeople have appealed to accept the judgeme...
1,"summarize: WATCH Maulana Asghar Ali Salafi, Pr...",\n\nAt a meeting at NSA Ajit Doval's residence...
2,summarize: A civilizational wound is now heale...,.\n\nThe AyodhyaVerdict has finally been reach...
3,summarize: As a step to impress Sonia Gandhian...,"\n\nThe tweets discuss the Ayodhya Verdict, a ..."
4,summarize: I would suggest Muslims of present ...,\n\nThis series of tweets discusses the recent...
...,...,...
1152,summarize: intutive9 This is very helpful to u...,\n\nA 19 year old Muthyam from Maharashtra die...
1153,"summarize: On Camera, 19YearOld Dancing At Wed...","OUS\n\nA 19-year-old man from Maharashtra, Mut..."
1154,summarize: I had doubt's on Covid vaccine from...,\n\nPeople have had doubts about the Covid-19 ...
1155,"summarize: anxiety frm pain, then hv docs,phar...",\n\nPeople are feeling overwhelmed with the am...


In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=1)

### Downloading t5 pre-trained model

In [7]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [15]:
model.device= torch.device("cpu")

## Calculating Rouge score without Fine Tuning

In [16]:
pred=[]
for i in tqdm(test['source_text'],total=len(test)):
    pred.append(model.predict(i))

100%|██████████| 116/116 [12:21<00:00,  6.40s/it]


In [17]:
ref_text=[]
for i in test['target_text']:
    ref_text.append(i)

In [20]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
#  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.264488,0.131064,0.204984,0.205316


# Fine Tuniing

In [14]:
model.train(train_df=train,
            eval_df=test, 
            source_max_token_len=1024, 
            target_max_token_len=150, 
            batch_size=4, max_epochs=5, use_gpu=True)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Inference

In [26]:
cd /kaggle/working/outputs

/kaggle/working/outputs


In [27]:
ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
simplet5-epoch-0-train-loss-1.6538-val-loss-1.2682/
simplet5-epoch-1-train-loss-1.2343-val-loss-1.1983/
simplet5-epoch-2-train-loss-1.0561-val-loss-1.1918/
simplet5-epoch-3-train-loss-0.9124-val-loss-1.1844/
simplet5-epoch-4-train-loss-0.8036-val-loss-1.2213/


In [32]:
test

,source_text,target_text
1130,summarize: Symptoms of heartattack may appear ...,stroke\n\nRecent research has found a link be...
202,summarize: DFS Awaiting Buy Signal based off 7...,"\n\nFive companies (DFS, DGII, DGX, DH, DHI, D..."
754,"summarize: ""Designed for your different moods....",\n\nLionel Messi and Argentina have been in th...
761,summarize: CapoDeSupremo Damn that's a lot of ...,\n\nLionel Messi has gifted 35 gold-plated iPh...
437,summarize: IPLAuction IPL2023Auction IPLMiniAu...,\n\nThe IPL Auction for 2022 and 2023 is under...
...,...,...
267,summarize: Apple iPhone Repair and Service Ap...,\n\nApple Support Golders Green London offers ...
370,summarize: Get ready to experience the next le...,\n\nGoogle BARD and ChatGPT are revolutionary ...
1075,summarize: بھارتی ادکارہ سشمیتا سین کو دل کا د...,recently revealed that she had a heart attack...
945,summarize: Where are the Aussies????? WPL2023S...,has been announced as the captain of Royal Ch...


## Loading the Fine Tuned Model

In [58]:
# let's load the trained model from the local output folder for inferencing:
model.load_model("t5","simplet5-epoch-3-train-loss-0.9124-val-loss-1.1844", use_gpu=True)

In [59]:
from tqdm import tqdm

### Predictions(summarys generated by finetuned model) are stored in pred

In [61]:
pred=[]
for i in tqdm(test['source_text'],total=len(test)):
    pred.append(model.predict(i))

100%|██████████| 116/116 [03:07<00:00,  1.61s/it]


In [54]:
ref_text=[]
for i in test['target_text']:
    ref_text.append(i)

In [21]:
from datasets import load_dataset, load_metric
rouge_metric = load_metric('rouge')

In [65]:
rouge_metric.add_batch(predictions=pred, references=ref_text)
        
#  Finally compute and return the ROUGE scores.
score = rouge_metric.compute()

In [66]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.493716,0.278843,0.36208,0.36294


### Rouge Score improved from 0.26 to 0.49